In [1]:
import os
import shutil
import subprocess
import qiime2 as q2
import pandas as pd
from biom import Table, load_table
from biom.util import biom_open
import glob
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


### running songbird

This is best done on a compute cluster but the scripts are provided in `diff-analysis-new/songbird-helper-new.py`. From these grid searches we determined the best perams. to run songbird with with a $Q^{2}$-value close to one. From these runs we can use the differentials to explore what microbes change between the birth-modes. 


#### Capture the grid search results at individual time points without country as covariate

In [4]:
%%capture
# for each body site repeat 
all_grid_results = {}
input_path = '../data/diff-analysis-new/songbird-grid-search/'
model_name = ['TP-w-training']
days = ["2.0", "30.0", "120.0", "180.0"]

for model_ in model_name:
    # get all body site path(s)
    partition_path = os.path.join(input_path, model_, 'Baby*.0')
    body_sites = [bs_.split('/')[-1]
                      for bs_ in glob.glob(partition_path)]
    # run for each body site
    for body_site_ in body_sites:
        # get all baseline models CV
        baseline_ls_path =  os.path.join(input_path, model_, body_site_, '1*')
        baseline_models = glob.glob(baseline_ls_path)
        # because of the maximum number of testing samples are set, 
        # sometimes it is not possible to try bigger batch_size,
        # therefore no output
        baseline_models = {tuple(id_.split('/')[-1].split('-')[1:]):id_
                           for id_ in baseline_models 
                           }
        # retrieve all baseline models
        for id_, path_ in baseline_models.items():
            # get path to data
            event_acc = EventAccumulator(path_)
            event_acc.Reload()
            # get scalar perams
            w_times, step_nums, vals = zip(*event_acc.Scalars('accuracy/cv_error'))
            baseline_models[id_] = [w_times, step_nums, vals]
        # get all fomrula based models CV
        all_ls_path =  os.path.join(input_path, model_, body_site_, '*')
        formula_models = glob.glob(all_ls_path)
        exclude_ = glob.glob(baseline_ls_path)
        formula_models = sorted(set(formula_models) - set(exclude_))
        formula_models = {tuple(id_.split('/')[-1].split('-')[:]):id_
                          for id_ in formula_models}
        formula_models = {('-'.join(k[:-4]),k[-4],k[-3],k[-2],k[-1]):v
                          for k,v in formula_models.items() 
                         }
        for id_, path_ in formula_models.items():
            # get path to data
            event_acc = EventAccumulator(path_)
            event_acc.Reload()
            # get scalar perams
            w_times, step_nums, vals = zip(*event_acc.Scalars('accuracy/cv_error'))
            # calc q^2-value
            base_cv = np.mean(baseline_models[id_[1:]][-1][-10:])
            form_cv = np.mean(vals[-10:])
            q_squared = 1 - form_cv/base_cv
            formula_models[id_] = [form_cv, base_cv, q_squared]
        # make dataframe to save
        gird_results = pd.DataFrame(formula_models).T.reset_index()
        gird_results.columns = ['formula', 'min_features', 'batch_size', 'traing_size',
                                'differential_prior', 'CV', 'baseline_CV',
                                'q_squared']
        all_grid_results[(model_, body_site_)] = gird_results


In [5]:
# all models
all_grid_df = pd.concat(all_grid_results).reset_index().drop('level_2', axis=1)
all_grid_df[['body_site', 'days']] = all_grid_df.level_1.str.rsplit(pat = "-", n = 1, expand = True)
all_grid_df['model'] = all_grid_df.level_0
# allowed models, aka. q2>0
all_grid_df_allowed = all_grid_df.drop(['level_0', 'level_1'], axis = 1)[all_grid_df.q_squared > 0].copy()
ind_ = all_grid_df_allowed.groupby(['body_site','days','formula'])[['CV']].idxmin().values
all_grid_df_allowed['best'] = np.nan
all_grid_df_allowed.loc[ind_.flatten(), 'best'] = 'Yes'
all_grid_df_allowed.to_csv('../data/diff-analysis-new/TP-w-training-grid-search-all-allowed-models3.tsv', sep='\t')
all_grid_df_allowed.loc[all_grid_df_allowed.best=='Yes', :]


,formula,min_features,batch_size,traing_size,differential_prior,CV,baseline_CV,q_squared,body_site,days,model,best
3,"C(birth_mode_ms, Treatment(""Vag"")) + country",2,4,7,0.5,231.073106,310.651822,0.256167,Baby-Mouth,120.0,TP-w-training,Yes
38,"C(birth_mode_ms, Treatment(""Vag""))",2,4,7,0.25,290.484720,310.650241,0.064914,Baby-Mouth,120.0,TP-w-training,Yes
84,"C(birth_mode_ms, Treatment(""Vag"")) + country",3,7,11,0.25,303.880743,350.733188,0.133584,Baby-Mouth,2.0,TP-w-training,Yes
110,"C(birth_mode_ms, Treatment(""Vag""))",3,11,15,0.25,451.999515,502.479327,0.100461,Baby-Mouth,2.0,TP-w-training,Yes
147,"C(birth_mode_ms, Treatment(""Vag"")) + country",2,4,6,0.5,125.618256,134.642320,0.067022,Baby-Forearm,180.0,TP-w-training,Yes
188,"C(birth_mode_ms, Treatment(""Vag""))",2,6,6,0.25,145.019194,146.240004,0.008348,Baby-Forearm,180.0,TP-w-training,Yes
226,"C(birth_mode_ms, Treatment(""Vag"")) + country",2,6,9,0.25,310.436407,542.532928,0.427802,Baby-Mouth,180.0,TP-w-training,Yes
256,"C(birth_mode_ms, Treatment(""Vag""))",2,4,9,0.25,280.060626,399.159140,0.298374,Baby-Mouth,180.0,TP-w-training,Yes
290,"C(birth_mode_ms, Treatment(""Vag"")) + country",2,4,6,0.25,33.109026,41.291887,0.198171,Baby-Forearm,120.0,TP-w-training,Yes
326,"C(birth_mode_ms, Treatment(""Vag""))",2,4,6,0.25,39.730360,41.291887,0.037817,Baby-Forearm,120.0,TP-w-training,Yes


In [6]:
file_path_ = '../data/diff-analysis-new/songbird-grid-search/'
file_path_copy_ = '../data/diff-analysis-new/songbird-optimized-models/' 

all_grid_df_best = all_grid_df_allowed.loc[all_grid_df_allowed.best=='Yes', :]

paths_copy_ = [(os.path.join(file_path_, md_, '-'.join([bs_, k_]),
                             '-'.join(df2_.values[0][0:5])),
                os.path.join(file_path_copy_, md_, '-'.join([bs_, k_]),
                             '-'.join(df2_.values[0][0:5])),
                os.path.join(file_path_, md_, '-'.join([bs_, k_]),
                             '1-' + '-'.join(df2_.values[0][1:5])),
                os.path.join(file_path_copy_, md_, '-'.join([bs_, k_]),
                             '1-' + '-'.join(df2_.values[0][1:5])))
               for md_, mddf_ in all_grid_df_best.groupby('model')
               for bs_, bsdf_ in mddf_.groupby('body_site')
               for k_, df_ in bsdf_.groupby('days')
               for k2_, df2_ in df_.groupby('formula')]
# copy the paths
for copy_ in paths_copy_:
    if not os.path.exists(copy_[1]):
        shutil.copytree(copy_[0], copy_[1])
        print(copy_[1])
    if not os.path.exists(copy_[3]):
        shutil.copytree(copy_[2], copy_[3])
        print(copy_[3])

../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-120.0/C(birth_mode_ms, Treatment("Vag"))-4-12-12-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-120.0/1-4-12-12-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-120.0/C(birth_mode_ms, Treatment("Vag")) + country-4-12-12-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-180.0/C(birth_mode_ms, Treatment("Vag"))-4-8-12-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-180.0/1-4-8-12-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-180.0/C(birth_mode_ms, Treatment("Vag")) + country-4-8-12-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-2.0/C(birth_mode_ms, Treatment("Vag"))-3-7-7-0.25
../data/diff-analysis-new/songbird-optimized-models/TP-w-training/Baby-Feces-2.0/1-3-7-7-0.25
../data/diff-analysis-new/songbird-optimiz

### Summarizing the results

In [2]:
import itertools
from skbio.stats.composition import (alr_inv, alr,
                                     closure, clr)
# warnings filter 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# import taxonomy
taxdf = q2.Artifact.load('../data/processed-data/taxonomy.qza'
                        ).view(q2.Metadata).to_dataframe()

def split_taxonomy(taxonomy):
    feat_map = dict(taxonomy.Taxon)
    taxonomy['Taxon'] = [feat_map[feat]
                         if feat in feat_map.keys()
                         else np.nan
                         for feat in taxonomy.index]
    # add taxonomic levels for grouping later (if available)

    def tax_split(tax_id, tax_level): return tax_id.split(
        tax_level)[1].split(';')[0]

    for level, lname in zip(['k__', 'p__', 'c__', 'o__',
                             'f__', 'g__', 's__'],
                            ['kingdom', 'phylum', 'class',
                             'order', 'family', 'genus',
                             'species']):
        if lname not in taxonomy.columns:
            taxonomy_tmp = []
            for tax in taxonomy.Taxon:
                if tax is not np.nan and\
                   level in tax and\
                   len(tax_split(tax, level)) > 0:
                    taxonomy_tmp.append(tax_split(tax,
                                                  level))
                else:
                    taxonomy_tmp.append(np.nan)
            taxonomy[lname] = taxonomy_tmp
    return taxonomy

# split the levels into columns
taxdf = split_taxonomy(taxdf)

In [4]:
def center_differentials(clr_diff):
    """ re-centers data around zero """
    
    # center again around zero after completion
    clr_diff = clr_diff \
                - clr_diff.mean(axis=0).values
    clr_diff = clr_diff \
                - clr_diff.mean(axis=1).values.reshape(-1, 1)
    # return the re-centered data
    return clr_diff

def differentials_to_probability(differentials,
                                 numerators,
                                 prefix='C(birth_mode, Treatment("CS"))',
                                 basis_name="P(CS)"):
    """ converts differentials to something like
        a probability using the inverse-alr transform.
    """
    # first recenter the differentials
    differentials = center_differentials(differentials)
    # then take the inverse alr
    prob_differentials = alr_inv(differentials[numerators])
    # make a dataframe to return 
    columns = [col.replace("[T.","P(").replace("]",")").replace(prefix, "")
                for col in numerators] # rename cols
    columns = [basis_name] + columns
    prob_differentials = pd.DataFrame(prob_differentials,
                                      differentials.index,
                                      columns)
    return prob_differentials

In [27]:
# container for differentials
all_differentials = {}
all_metadata = {}
all_tables = {}
# get path info
data_path = '../data/diff-analysis-new'
model_name = ['TP-w-training']
diffs_use = ['[T.CS]', '[T.CSseed]']

for model_ in model_name:
    input_path = ('../data/diff-analysis-new/songbird-optimized-models/' 
                  + model_)
    # add a frequency filter
    min_freq = 0.0
    # get all body site path(s)
    partition_path = os.path.join(input_path, 'Baby*')
    body_sites = [bs_.split('/')[-1]
                      for bs_ in glob.glob(partition_path)]
    # run for each body site
    for body_site_ in body_sites:
        # for body site subsets
        sub_path = os.path.join(input_path, body_site_, '*')

        # get differentials
        baseline_ls_path = os.path.join(input_path, body_site_, '1*')    
        formula_models = glob.glob(sub_path)
        exclude_ = glob.glob(baseline_ls_path)
        formula_models = sorted(set(formula_models) - set(exclude_))
        # get diffs. and add taxonomy labels
        for formula_ in formula_models:
            diff = pd.read_csv(os.path.join(formula_, 'differentials.tsv'),
                               sep='\t', index_col=0)
            # get table and metadata for each subset
            data_split_path = os.path.join(data_path, body_site_)
            mf = pd.read_csv(os.path.join(data_split_path,
                                          'metadata.tsv'),
                               sep='\t', index_col=0)
            bt = load_table(os.path.join(data_split_path,
                                         'table.biom'))
            # caclulate the freq. of that feature in the data, the number of times a feature appeared
            # in the sampels
            frequncy = pd.DataFrame(bt.matrix_data.toarray().astype(bool).sum(1) / 
                                    bt.shape[1], 
                                    bt.ids('observation'), ['feature-frequency']) 
            frequncy = frequncy.reindex(diff.index)
            # add to diff to reduce files saved
            diff['feature-frequency'] = frequncy['feature-frequency']
            # apply the filter
            diff = diff[diff['feature-frequency'] >= min_freq]
            # reindex taxonomy
            taxdf_ = taxdf.reindex(diff.index)
            # get the prob. of each microbe in each state
            differential_cols = [col_ for col_ in diff.columns
                                 if any(dc in col_ for dc in diffs_use)]
            diff_ = center_differentials(diff[['Intercept'] + differential_cols])
            diff_.columns = ['centered-' + 
                             hh.replace("[T", "diff").replace("]", "").replace('C(birth_mode_ms, Treatment("Vag"))', "") 
                             for hh in diff_.columns]

            pdiff = differentials_to_probability(diff[['Intercept'] + 
                                                      differential_cols],
                                                 differential_cols, 
                                                 prefix = 'C(birth_mode_ms, Treatment("Vag"))',
                                                 basis_name="P(Vag)")
            pdiff = pdiff.reindex(diff.index)
            diff.columns = [hh.replace('C(birth_mode_ms, Treatment("Vag"))', "") 
                             for hh in diff.columns]
            # add all together
            diff = pd.concat([pdiff, diff, diff_, taxdf_], axis=1)
            # calculate seeding effectiveness metric TBA
            diff['seeding-effectiveness'] = (diff['P(Vag)'] * diff['P(CSseed)']) + (diff['P(CS)']/4)
            # save the files
            all_differentials[(model_, body_site_, formula_.split('/')[-1])] = diff.rename({'P(CSseed)':'P(CS-seeded)',
                                                                    'P(Vag)':'P(Vaginal)'}, axis=1)
        all_metadata[(model_, body_site_)] = mf
        all_tables[(model_, body_site_)] = bt



In [29]:
pd.concat(all_differentials).reset_index().rename(columns = {'level_0':'model', 
                                                             'level_1':'body_site_days',
                                                            'level_2': 'model-para'}).to_csv(
    os.path.join(data_path, 'TP-w-training-best-model-processed.tsv'), sep='\t')